In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# GLOBAL VARIABLES

dfModelSegSummaries = pd.DataFrame( [
    [2019, 'data/model-output/v9_SE19_Net19_Summary_SEGID.csv'    ],
    [2028, 'data/model-output/v9_TIP_SE28_Net28_Summary_SEGID.csv'],
    [2023, 'data/model-output/v9_SE23_Net23_Summary_SEGID.csv'    ],
    [2032, 'data/model-output/v9_RTP_SE32_Net32_Summary_SEGID.csv'],
    [2042, 'data/model-output/v9_RTP_SE42_Net42_Summary_SEGID.csv'],
    [2050, 'data/model-output/v9_RTP_SE50_Net50_Summary_SEGID.csv'],
], columns=('modYear','modSegSummaryFile'))

#display(dfModelSegSummaries)

# seg summary fields to be useds
use_fields = ['SEGID','DY_VOL']

# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
fnSegmentsShapefile = 'data/segments/Segments_WF - 2023-08-01.shp'

# field for weekday factor
colWeekdayFactor = 'FAC_WDAVG'

# SHOULD WE ADJUST WEEKDAY FACTOR INTO FUTURE?? 

# base year adjustment
# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
sourceName = 'Segments_WF - 2023-08-01.shp'
baseYear = 2019



In [3]:
# READ IN CSVs and STORE
# ChatGPT: https://chat.openai.com/share/8d8aa536-ad1f-4893-a6dc-3e112201beee

# Create a list to store DataFrames read from each CSV
frames = []

# Iterate through the rows and read each CSV
for index, row in dfModelSegSummaries.iterrows():
    df = pd.read_csv(row['modSegSummaryFile'])
    df = df[use_fields]
    df['YEAR'] = row['modYear'] # Add modYear column
    frames.append(df)

# Concatenate all the frames into a single DataFrame
dfMdlVol = pd.concat(frames, ignore_index=True)

display(dfMdlVol)

,SEGID,DY_VOL,YEAR
0,0006_146.9,1510.7,2019
1,0006_149.9,1559.7,2019
2,0006_150.6,1755.8,2019
3,0006_152.6,2495.6,2019
4,0006_152.9,3725.4,2019
...,...,...,...
23002,WFRC_8261,18.9,2050
23003,WFRC_8263,7783.9,2050
23004,WFRC_8264,12940.8,2050
23005,WFRC_8265,4036.0,2050


In [4]:
# GET WEEKDAY FACTORS

# read in segment shapefile
gdfSegments = gpd.read_file(fnSegmentsShapefile)
dfWkDyFac = gdfSegments[['SEGID', colWeekdayFactor]]

display(dfWkDyFac)

,SEGID,FAC_WDAVG
0,0006_000.0,0.984
1,0006_000.7,0.984
2,0006_016.0,0.984
3,0006_046.0,0.984
4,0006_060.2,0.984
...,...,...
8722,UTA_7132,0.000
8723,UTA_7316,0.000
8724,UTA_7320,0.000
8725,UTA_7328,0.000


In [5]:
import numpy as np

# Your existing code for merging and filling NA
_df = pd.DataFrame.merge(dfMdlVol, dfWkDyFac, on='SEGID', how='left')
_df.fillna(0, inplace=True)

# Create a mask where the denominator is zero
mask_zero_denominator = _df[colWeekdayFactor] == 0

# Apply the division only where the denominator is not zero, and set the value to 0 where it is zero
_df['modAADT'] = np.where(mask_zero_denominator, 0, _df['DY_VOL'] / _df[colWeekdayFactor])

# Convert to integer as you had before
_df['modAADT'] = _df['modAADT'].astype(int)

dfModAadt = _df

# Display the DataFrame
display(dfModAadt)

,SEGID,DY_VOL,YEAR,FAC_WDAVG,modAADT
0,0006_146.9,1510.7,2019,0.9840,1535
1,0006_149.9,1559.7,2019,0.9840,1585
2,0006_150.6,1755.8,2019,0.9840,1784
3,0006_152.6,2495.6,2019,0.9840,2536
4,0006_152.9,3725.4,2019,0.9593,3883
...,...,...,...,...,...
23026,WFRC_8261,18.9,2050,0.0000,0
23027,WFRC_8263,7783.9,2050,1.0924,7125
23028,WFRC_8264,12940.8,2050,1.0924,11846
23029,WFRC_8265,4036.0,2050,1.0924,3694


In [6]:
# SHOW WHERE FAC_WDAVG is 0... KEEP ZERO UNTIL FIGURE OUT WHAT TO DO!
dfModAadt[dfModAadt[colWeekdayFactor]==0]


,SEGID,DY_VOL,YEAR,FAC_WDAVG,modAADT
38,0015_250.3,22163.0,2019,0.0,0
798,0134_007.8,3478.9,2019,0.0,0
1057,0194_002.1,18556.6,2019,0.0,0
1560,1508_000.4,8032.2,2019,0.0,0
1612,2036_003.1,13985.2,2019,0.0,0
...,...,...,...,...,...
23023,WFRC_8258,3698.5,2050,0.0,0
23024,WFRC_8259,7257.0,2050,0.0,0
23025,WFRC_8260,7295.2,2050,0.0,0
23026,WFRC_8261,18.9,2050,0.0,0


In [7]:
_df = pd.read_csv('intermediate/aadt.csv')
_df

,SEGID,YEAR,AADT,SOURCE
0,0006_146.9,1981,975,AADTHistory.xlsx
1,0006_146.9,1982,1000,AADTHistory.xlsx
2,0006_146.9,1983,965,AADTHistory.xlsx
3,0006_146.9,1984,1240,AADTHistory.xlsx
4,0006_146.9,1985,1340,AADTHistory.xlsx
...,...,...,...,...
159921,WFRC_8263,2015,13770,Segments_WF - 2023-08-01.shp
159922,WFRC_8263,2016,14498,Segments_WF - 2023-08-01.shp
159923,WFRC_8263,2017,6489,Segments_WF - 2023-08-01.shp
159924,WFRC_8263,2018,6554,Segments_WF - 2023-08-01.shp


In [8]:
# GET BASE YEAR OBSERVED

# read intermediate CSV with base year observed AADT
_df = pd.read_csv('intermediate/aadt.csv')

# filter by source and year
_df = _df[(_df['SOURCE']==sourceName) & (_df['YEAR']<=baseYear)]

# get max row of max year for each segid... incase there is no observed 2019 data, but maybe a earlier year
# Then, group by 'SEGID' and find the index of the maximum 'YEAR' for each group
idx = _df.groupby('SEGID')['YEAR'].idxmax()

# Use the index to filter the rows
_df = _df.loc[idx]

_df = _df[['SEGID','AADT']]
_df.rename(columns={'AADT':'obsAADT'}, inplace=True)

dfObsAadt_BaseYear = _df

display(dfObsAadt_BaseYear)

,SEGID,obsAADT
78,0006_146.9,1517
157,0006_149.9,2441
236,0006_150.6,2441
313,0006_152.6,2417
390,0006_152.9,3759
...,...,...
159903,WFRC_8221,3
159904,WFRC_8222,3
159905,WFRC_8223,3
159915,WFRC_8262,6672


In [9]:
# CALCULATE ADJUSTMENT FACTOR FROM BASE YEAR MODEL VS OBSERVED
dfModAadt_BaseYear = dfModAadt[dfModAadt['YEAR']==2019]

_df = pd.DataFrame.merge(dfModAadt_BaseYear[['SEGID','modAADT']], dfObsAadt_BaseYear[['SEGID','obsAADT']], on='SEGID', how='left')
_df.fillna(0,inplace=True)
_df['aadtAdjFactor'] = _df['obsAADT'] - _df['modAADT']

_df.loc[(_df['modAADT']==0), 'aadtAdjFactor'] = 0

display(_df[_df['modAADT']==0])

dfModAdjFactor = _df[['SEGID','aadtAdjFactor']]

display(dfModAdjFactor)


,SEGID,modAADT,obsAADT,aadtAdjFactor
38,0015_250.3,0,0.0,0.0
798,0134_007.8,0,0.0,0.0
807,0135_000.1,0,0.0,0.0
1057,0194_002.1,0,0.0,0.0
1106,0202_001.5,0,884.0,0.0
...,...,...,...,...
3720,WFRC_8258,0,0.0,0.0
3721,WFRC_8259,0,0.0,0.0
3722,WFRC_8260,0,0.0,0.0
3723,WFRC_8261,0,0.0,0.0


,SEGID,aadtAdjFactor
0,0006_146.9,-18.0
1,0006_149.9,856.0
2,0006_150.6,657.0
3,0006_152.6,-119.0
4,0006_152.9,-124.0
...,...,...
3721,WFRC_8259,0.0
3722,WFRC_8260,0.0
3723,WFRC_8261,0.0
3724,WFRC_8263,2427.0


In [10]:
# ADJUST ALL MODEL VOLUMES BASED ON BASE YEAR ADJ FACTOR

dfModWithAdj = pd.DataFrame.merge(dfModAadt, dfModAdjFactor, on='SEGID', how='left')
dfModWithAdj['modForecast'] = dfModWithAdj['modAADT'] + dfModWithAdj['aadtAdjFactor']
dfModWithAdj

,SEGID,DY_VOL,YEAR,FAC_WDAVG,modAADT,aadtAdjFactor,modForecast
0,0006_146.9,1510.7,2019,0.9840,1535,-18.0,1517.0
1,0006_149.9,1559.7,2019,0.9840,1585,856.0,2441.0
2,0006_150.6,1755.8,2019,0.9840,1784,657.0,2441.0
3,0006_152.6,2495.6,2019,0.9840,2536,-119.0,2417.0
4,0006_152.9,3725.4,2019,0.9593,3883,-124.0,3759.0
...,...,...,...,...,...,...,...
23074,WFRC_8261,18.9,2050,0.0000,0,0.0,0.0
23075,WFRC_8263,7783.9,2050,1.0924,7125,2427.0,9552.0
23076,WFRC_8264,12940.8,2050,1.0924,11846,NaN,NaN
23077,WFRC_8265,4036.0,2050,1.0924,3694,NaN,NaN


In [11]:
# check if base year adjusted equals base year observed

_df1 = dfModWithAdj[dfModWithAdj['YEAR']==baseYear]
_df2 = dfObsAadt_BaseYear

_df3 = pd.DataFrame.merge(_df1, _df2, on='SEGID')

_df3['ModVsObs'] = _df3['modForecast'] - _df3['obsAADT']

display(_df3[_df3['ModVsObs']!=0])


,SEGID,DY_VOL,YEAR,FAC_WDAVG,modAADT,aadtAdjFactor,modForecast,obsAADT,ModVsObs
1096,0202_001.5,0.0,2019,0.9840,0,0.0,0.0,884,-884.0
1786,2082_005.0,0.0,2019,1.0918,0,0.0,0.0,9697,-9697.0
2562,2372_000.0,0.0,2019,1.0946,0,0.0,0.0,4123,-4123.0
2956,2964_003.0,0.0,2019,1.0924,0,0.0,0.0,5257,-5257.0


In [12]:
# export final file
dfModWithAdj.to_csv('intermediate/model-forecasts.csv', index=False)